<a href="https://colab.research.google.com/github/linshaochieh2019/test/blob/main/ventilator_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import time
from datetime import datetime

import torch
from torch.utils.data import random_split, DataLoader, Dataset

In [24]:
train_csv = '/content/drive/MyDrive/Data/ventilator1/train.csv'

train = pd.read_csv(train_csv)

print('# data entries: {}'.format(len(train)))
print('# breath id: {}'.format(len(train['breath_id'].unique())))

# data entries: 6036000
# breath id: 75450


In [25]:
def evenly_spaced(df, breath_id):
  
  df_temp = df[df['breath_id'] == breath_id]
  df_temp = df_temp[['time_step', 'pressure', 'u_in', 'u_out']]
  df_temp['mask'] = np.zeros(len(df_temp))

  data_temp = {'time_step': np.arange(0,2.5,0.025),
               'pressure': np.nan,
               'u_in': np.nan,
               'u_out': np.nan,
               'mask':np.ones(len(np.arange(0,2.5,0.025)))}
  
  df_gen = pd.DataFrame.from_dict(data_temp)
  df_new = pd.concat((df_gen, df_temp))
  df_new = df_new.sort_values(by='time_step', ascending=True)
  df_new = df_new.interpolate(method='linear', axis=0)
  df_new = df_new.iloc[1:] #remove time step == 0
  df_new = df_new[df_new['mask'] == 1] #remove old time steps
  df_new = df_new.drop(['mask'], axis=1) #remove mask column

  temp_dict = df_new.to_dict(orient='list')
  temp_dict['breath_id'] = np.ones(len(df_new)) * breath_id

  return temp_dict

In [26]:
data_dict = {
              'breath_id': [],
              'pressure': [],
              'u_in': [],
              'u_out': [],
              'time_step': [],
             }

ls_cols = ['breath_id', 'pressure', 'u_in', 'u_out', 'time_step']

ls_breath_id = train['breath_id'].unique()
for breath_id in tqdm(ls_breath_id):
  data_temp = evenly_spaced(df=train, breath_id=breath_id)
  for col in ls_cols:
    data_dict[col].extend(data_temp[col])

 36%|███▌      | 27112/75450 [07:02<12:52, 62.61it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
 39%|███▊      | 29205/75450 [07:35<11:53, 64.85it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
 39%|███▉      | 29723/75450 [07:44<12:08, 62.75it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [29]:
df = pd.DataFrame(data_dict)

# Convert breath_id to string
df['breath_id'] = df['breath_id'].astype(int).astype(str)

# Convert u_out to integer
df['u_out'] = df['u_out'].astype(int)

df.to_csv('/content/drive/MyDrive/Data/ventilator1/df_prep.csv')